In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import json
import requests
import pandas as pd
from tqdm.notebook import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [ ]:
df1 = pd.read_excel("S&P 500 Index.xlsx",header=None)
df1.shape

In [ ]:
new_columns = ["index", "code", "name"]
df1[new_columns] = df1.apply(lambda x: x[0].split(" "), axis = 1, result_type = "expand")
df1.shape

In [ ]:
stock_list = df1.code.to_list()
stock_list[:5]

In [ ]:
len(stock_list)

In [ ]:
api_key = 'c47a10c4dd90f0f9382ce2fdf0f59c94'  # You may find your api key here https://site.financialmodelingprep.com/developer/docs/api-keys

In [ ]:
all = pd.DataFrame()
for stock in tqdm(stock_list):
    for page in tqdm(range(500)):
        url = f"https://financialmodelingprep.com/api/v3/stock_news?tickers={stock}&page={page+1}&apikey={api_key}"
        s = requests.Session()

        retries = Retry(total=50,
                        status_forcelist=[429, 443, 500, 502, 503, 504])
        s.mount('http://', HTTPAdapter(max_retries=retries))
        res = s.get(url)
        res = json.loads(res.text)
        if len(res) == 0:
            break
        else:
            res = pd.DataFrame(res,index=[0])
            all = pd.concat([all, res])

In [ ]:
all.shape

In [ ]:
all = all.reset_index(drop=True)

In [ ]:
all

In [ ]:
all.to_csv("dataset.csv")

In [ ]:
all.symbol.nunique()

In [ ]:
all.groupby("symbol").publishedDate.min().to_csv("last_date.csv")

In [ ]:
all.head(2)

In [ ]:
res.url[0]